# Quickstart Notebook - Luminary Cloud Python SDK

To use this notebook, simply run each cell in order.

> NOTE: If you haven’t used the Luminary web app yet, you will need to [log in](https://app.luminarycloud.com/) and accept the terms and conditions before proceeding.


### Import the SDK

In [1]:
# import the Luminary Cloud SDK
import luminarycloud as lc

# import other misc dependencies
import os
import pandas as pd

### Create a new project

We'll start by creating a project to work in.

> **NOTE:** If this is the first time you're using the notebook, an authentication link
will be printed when you run the code cell below. 
>
> Click the link to open the login page in a new tab and use the same
credentials as the Luminary Cloud web app. Once successful, you can close the
tab and return to the notebook. The API call should have succeeded and you can
continue with the rest of the notebook.

In [ ]:
project = lc.create_project("My Project Name")
print(f"Created new project {project.id} named '{project.name}'")

### List all projects

We can also list all the projects in our account.

In [ ]:
for proj in lc.list_projects():
    print(proj)
    print("------------")

### Upload a new mesh

Now that we have a project, let's upload a mesh to the project. We'll use the mesh and simulation parameters from this sample project: https://docs.luminarycloud.com/en/articles/9396448-naca-0012-airfoil

In [ ]:
mesh = project.upload_mesh("./data/airfoil.lcmesh")
print(mesh)

### Create new simulation with uploaded mesh
Next, we'll create a simulation with the mesh we just uploaded. We'll use a simulation configuration downloaded from the UI.

> **NOTE:** to download the params from the UI, login and open a project. Then, click `...` next to the `Upload` button in the main settings panel. In the dropdown, select `Download simulation parameters JSON (API use only)`.

In [ ]:
simulation_template = project.create_simulation_template(
    name="SDK Sim Template", params_json_path="./data/api_simulation_params.json"
)
simulation = project.create_simulation(
    mesh.id, "Airfoil Simulation", simulation_template_id=simulation_template.id
)  # sim name
print(simulation)

The simulation is now running!

Let's wait for it to finish using `simulation.wait()`:

In [ ]:
status = simulation.wait()
print("Simulation finished with status:", status.name)

### Post-processing

Now that the simulation is done, let's download and inspect the residuals.

In [ ]:
# see documentation for more details about optional parameters
with simulation.download_global_residuals() as stream:
    residuals_df = pd.read_csv(stream, index_col="Iteration index")
    # since this is a steady state simulation, we can drop these columns
    residuals_df = residuals_df.drop(["Time step", "Physical time"], axis=1)

residuals_df

Here we plot the residuals, similar to what we see in the UI.

In [ ]:
ax = residuals_df.plot(logy=True, figsize=(12, 8))

In addition to residuals, the SDK also supports additional outputs. Lets download and inspect the
Lift output next.

> **NOTE:** the surface names in the request below come from the mesh. We will
soon have an API call to allow SDK users to see the available surface names in
the mesh.

In [ ]:
import luminarycloud as lc
from luminarycloud.enum import QuantityType, ReferenceValuesType

ref_vals = lc.ReferenceValues(
    reference_value_type=ReferenceValuesType.PRESCRIBE_VALUES,
    area_ref=1.0,
    length_ref=1.0,
    p_ref=101325.0,
    t_ref=273.15,
    v_ref=265.05709547039106,
)

# see documentation for more details about optional parameters
with simulation.download_surface_output(
    QuantityType.LIFT, ["0/bound/airfoil"], frame_id="body_frame_id", reference_values=ref_vals
) as stream:
    lift_df = pd.read_csv(stream, index_col="Iteration index")
    # since this is a steady state simulation, we can drop these columns
    lift_df = lift_df.drop(["Time step", "Physical time"], axis=1)

lift_df

In [ ]:
ax = lift_df.plot(figsize=(12, 8))

Lastly, we can download the surface and volume solutions for external post-processing with Paraview:

In [ ]:
# see documentation for more details about optional parameters
latest_solution = simulation.list_solutions()[-1]
with latest_solution.download_surface_data() as streaming_tar_file:
    path = f"./surface_data_{simulation.id}"
    streaming_tar_file.extractall(path)
    print(f"Extracted files to {path}:")
    print("\t" + "\n\t".join(os.listdir(path)))

In [ ]:
with latest_solution.download_volume_data() as streaming_tar_file:
    path = f"./volume_solution_{simulation.id}"
    streaming_tar_file.extractall(path)
    print(f"Extracted files to {path}:")
    for root, dirs, filenames in os.walk(path):
        print("\t" + "\n\t".join([os.path.join(root, file) for file in filenames]))